In [8]:
pip install scipy arviz numpy matplotlib pymc3 seaborn pandas


     |████████████████████████████████| 27.7MB 2.1MB/s eta 0:00:01
     |████████████████████████████████| 1.4MB 2.3MB/s eta 0:00:01
     |████████████████████████████████| 860kB 2.5MB/s eta 0:00:01
     |████████████████████████████████| 215kB 2.5MB/s eta 0:00:01
     |████████████████████████████████| 604kB 2.3MB/s eta 0:00:01
     |████████████████████████████████| 6.4MB 2.4MB/s eta 0:00:01
     |████████████████████████████████| 2.8MB 2.1MB/s eta 0:00:01
     |████████████████████████████████| 235kB 2.2MB/s eta 0:00:01
     |████████████████████████████████| 3.0MB 2.1MB/s eta 0:00:01
     |████████████████████████████████| 563kB 2.5MB/s eta 0:00:01
  Stored in directory: /Users/stewart/Library/Caches/pip/wheels/88/fb/be/483910ff7e9f703f30a10605ad7605f3316493875c86637014
Successfully built theano
Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install sklearn

     |████████████████████████████████| 10.5MB 6.1MB/s eta 0:00:01
     |████████████████████████████████| 286kB 1.2MB/s eta 0:00:01
  Stored in directory: /Users/stewart/Library/Caches/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Successfully built sklearn
Note: you may need to restart the kernel to use updated packages.


In [16]:
from scipy import stats
import arviz as az
import numpy as np
import matplotlib.pyplot as plt
import pymc3 as pm
import seaborn as sns
import pandas as pd
from theano import shared
from sklearn import preprocessing
print('Running on PyMC3 v{}'.format(pm.__version__))
data = pd.read_csv('renfe.csv')
data.drop(data.columns[[0]], axis = 1)
data = data.sample(frac=0.01, random_state=99)
data.head(3)

Running on PyMC3 v3.7


,insert_date,origin,destination,start_date,end_date,train_type,price,train_class,fare
4738,2019-04-17 08:03:35,VALENCIA,MADRID,2019-05-21 08:00:00,2019-05-21 09:47:00,AVE,33.65,Turista,Promo
8722,2019-05-09 07:12:41,MADRID,BARCELONA,2019-05-25 08:30:00,2019-05-25 11:15:00,AVE,85.10,Turista,Promo
13185,2019-04-24 11:21:26,MADRID,VALENCIA,2019-05-22 17:40:00,2019-05-22 19:20:00,AVE,51.15,Turista,Promo
